In [1]:
import argparse
import time
import tiktoken
import torch
import torch.nn as nn

In [23]:
GPT_CONFIG_124M = {
        "vocab_size": 50257,     # Vocabulary size
        "context_length": 1024,  # Context length
        "emb_dim": 768,          # Embedding dimension
        "n_heads": 12,           # Number of attention heads
        "n_kv_heads": 4,         #
        "n_layers": 12,          # Number of layers
        "drop_rate": 0.1,        # Dropout rate
        "qkv_bias": False        # Query-Key-Value bias
    }

In [24]:
class GroupedQueryAttention(nn.Module):
    def __init__(self, d_in, d_out, dropout, num_heads, num_kv_groups,  qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        assert num_heads % num_kv_groups == 0, "num_heads must be divisible by num_kv_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.num_kv_groups = num_kv_groups
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim
        self.group_size = num_heads // num_kv_groups

        self.W_query =  nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, self.num_kv_groups * self.head_dim, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, self.num_kv_groups * self.head_dim, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)

        ####################################################
        # KV cache-related code
        self.register_buffer("cache_k", None, persistent=False)
        self.register_buffer("cache_v", None, persistent=False)
        self.ptr_current_pos = 0
        ####################################################

    def forward(self, x, use_cache=False):
        b, num_tokens, d_in = x.shape

        keys_new = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        values_new = self.W_value(x)
        queries = self.W_query(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys_base = keys_new.view(b, num_tokens, self.num_kv_groups, self.head_dim).transpose(1, 2)
        values_base = values_new.view(b, num_tokens, self.num_kv_groups, self.head_dim).transpose(1, 2)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)

        ####################################################
        # KV cache-related
        if use_cache:
            if self.cache_k is None:
                self.cache_k, self.cache_v = keys_base, values_base
            else:
                self.cache_k = torch.cat([self.cache_k, keys_base], dim=1)
                self.cache_v = torch.cat([self.cache_v, values_base], dim=1)
            keys, values = self.cache_k, self.cache_v
        else:
            keys, values = keys_base, values_base
        ####################################################
        keys = torch.repeat_interleave(keys, self.group_size, dim = 1)
        values = torch.repeat_interleave(values, self.group_size, dim = 1)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        print(f"{queries.shape=}, {keys.shape=}, {values.shape=}")
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head


        ####################################################
        # causal mask
        num_tokens_Q = queries.shape[-2]
        num_tokens_K = keys.shape[-2]
        device = queries.device
        if use_cache:
            q_positions = torch.arange(
                self.ptr_current_pos,
                self.ptr_current_pos + num_tokens_Q,
                device=device,
                dtype=torch.long,
            )
            self.ptr_current_pos += num_tokens_Q
        else:
            q_positions = torch.arange(num_tokens_Q, device=device, dtype=torch.long)
            self.ptr_current_pos = 0
        k_positions = torch.arange(num_tokens_K, device=device, dtype=torch.long)
        mask_bool = q_positions.unsqueeze(-1) < k_positions.unsqueeze(0)

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

    def reset_cache(self):
        self.cache_k, self.cache_v = None, None
        self.ptr_current_pos = 0

In [15]:
torch.manual_seed(123)
gqa = GroupedQueryAttention(8, 8, 0.1, 4, 2)
x = torch.randn((4, 16, 8))
gqa(x, use_cache=True)


queries.shape=torch.Size([4, 4, 16, 2]), keys.shape=torch.Size([4, 4, 16, 2]), values.shape=torch.Size([4, 4, 16, 2])


tensor([[[ 0.2920,  0.0703,  0.2762, -0.2524, -0.5075, -0.1266,  0.3727,
           0.6040],
         [ 0.2058,  0.1342, -0.0123, -0.4686, -0.2325, -0.2556,  0.4262,
           0.5283],
         [ 0.2384,  0.1692, -0.0439, -0.3199, -0.2742, -0.2083,  0.4479,
           0.4003],
         [ 0.3033,  0.1678, -0.0886, -0.1825,  0.0342, -0.1634,  0.2930,
           0.3305],
         [ 0.1602,  0.2293, -0.1543, -0.1237,  0.0383, -0.2116,  0.2886,
           0.1665],
         [ 0.0709,  0.3052, -0.2888, -0.2147,  0.2245, -0.3072,  0.2445,
           0.1013],
         [ 0.0150,  0.3179, -0.2837, -0.0542,  0.2898, -0.2506,  0.1499,
          -0.0657],
         [ 0.0024,  0.3060, -0.3070,  0.0418,  0.3283, -0.2031,  0.1362,
          -0.1713],
         [ 0.0414,  0.2747, -0.2766, -0.0530,  0.3010, -0.3000,  0.2065,
           0.0594],
         [ 0.0709,  0.2530, -0.2590, -0.0353,  0.4087, -0.3184,  0.1563,
           0.0994],
         [-0.0824,  0.4708, -0.5199,  0.0191,  0.3002, -0.1633,  0.147

In [16]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [17]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [18]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [21]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = GroupedQueryAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            num_heads=cfg["n_heads"],
            num_kv_groups=cfg["n_kv_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x, use_cache=False):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)

        # x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        ####################################################
        #  KV cache-related
        x = self.att(x, use_cache=use_cache)
        ####################################################

        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x


In [26]:
torch.manual_seed(123)
trb = TransformerBlock(cfg = GPT_CONFIG_124M)
x = torch.randn((4, 16, 768))
trb(x, use_cache=True)

queries.shape=torch.Size([4, 12, 16, 64]), keys.shape=torch.Size([4, 12, 16, 64]), values.shape=torch.Size([4, 12, 16, 64])


tensor([[[ 3.6094e-01, -1.2579e+00, -9.9960e-01,  ..., -9.0474e-01,
          -6.4033e-01, -6.8059e-01],
         [-1.1922e+00, -2.3845e+00, -2.7026e-02,  ...,  1.3562e+00,
          -1.1489e+00, -1.3625e+00],
         [-1.5047e-01, -5.6593e-01, -1.2998e+00,  ...,  4.3289e-01,
          -7.0673e-01, -3.1272e-01],
         ...,
         [ 7.3964e-01, -8.9510e-01, -3.6306e-01,  ..., -1.3957e+00,
          -4.7772e-01,  2.5455e+00],
         [ 1.8428e+00, -1.9542e+00, -8.9446e-01,  ..., -1.4467e+00,
          -3.8625e-01,  4.6340e-01],
         [-1.1149e+00, -5.9360e-01, -3.8470e-01,  ...,  3.4420e-01,
          -1.3897e+00,  9.8030e-01]],

        [[-1.2351e-01,  6.8971e-01,  1.2014e-01,  ...,  2.8277e-01,
           1.5974e+00,  6.3322e-01],
         [ 9.2666e-02,  1.7620e+00,  2.4819e-01,  ...,  8.9675e-01,
           1.7424e-01, -3.7959e-01],
         [-3.2083e-01,  3.5708e-01, -6.6473e-01,  ...,  2.0882e-01,
           6.1683e-01,  3.0177e+00],
         ...,
         [-7.0430e-01, -1

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = GroupedQueryAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            num_heads=cfg["n_heads"],
            num_kv_groups=cfg["n_kv_groups"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x, use_cache=False):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)

        # x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        ####################################################
        #  KV cache-related
        x = self.att(x, use_cache=use_cache)
        ####################################################

        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [27]:
torch.manual_seed(123)
trb = TransformerBlock(cfg = GPT_CONFIG_124M)
x = torch.randn((4, 16, 768))
trb(x, use_cache=True)

queries.shape=torch.Size([4, 12, 16, 64]), keys.shape=torch.Size([4, 12, 16, 64]), values.shape=torch.Size([4, 12, 16, 64])


tensor([[[ 3.6094e-01, -1.2579e+00, -9.9960e-01,  ..., -9.0474e-01,
          -6.4033e-01, -6.8059e-01],
         [-1.1922e+00, -2.3845e+00, -2.7026e-02,  ...,  1.3562e+00,
          -1.1489e+00, -1.3625e+00],
         [-1.5047e-01, -5.6593e-01, -1.2998e+00,  ...,  4.3289e-01,
          -7.0673e-01, -3.1272e-01],
         ...,
         [ 7.3964e-01, -8.9510e-01, -3.6306e-01,  ..., -1.3957e+00,
          -4.7772e-01,  2.5455e+00],
         [ 1.8428e+00, -1.9542e+00, -8.9446e-01,  ..., -1.4467e+00,
          -3.8625e-01,  4.6340e-01],
         [-1.1149e+00, -5.9360e-01, -3.8470e-01,  ...,  3.4420e-01,
          -1.3897e+00,  9.8030e-01]],

        [[-1.2351e-01,  6.8971e-01,  1.2014e-01,  ...,  2.8277e-01,
           1.5974e+00,  6.3322e-01],
         [ 9.2666e-02,  1.7620e+00,  2.4819e-01,  ...,  8.9675e-01,
           1.7424e-01, -3.7959e-01],
         [-3.2083e-01,  3.5708e-01, -6.6473e-01,  ...,  2.0882e-01,
           6.1683e-01,  3.0177e+00],
         ...,
         [-7.0430e-01, -1

In [ ]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # self.trf_blocks = nn.Sequential(
        #    *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        ####################################################
        #  KV cache-related
        self.trf_blocks = nn.ModuleList(
            [TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.current_pos = 0
        ####################################################

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx, use_cache=False):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)

        # pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))

        ####################################################
        #  KV cache-related
        if use_cache:
            pos_ids = torch.arange(self.current_pos, self.current_pos + seq_len, device=in_idx.device, dtype=torch.long)
            self.current_pos += seq_len
        else:
            pos_ids = torch.arange(0, seq_len, device=in_idx.device, dtype=torch.long)
        pos_embeds = self.pos_emb(pos_ids).unsqueeze(0)
        ####################################################

        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)

        # x = self.trf_blocks(x)
        ####################################################
        # KV cache-related
        for blk in self.trf_blocks:
            x = blk(x, use_cache=use_cache)
        ####################################################

        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

    ####################################################
    # KV cache-related
    def reset_kv_cache(self):
        for blk in self.trf_blocks:
            blk.att.reset_cache()
        self.current_pos = 0
    ####################################################